# 자연어 처리: 텍스트 벡터화 구현

## 개요
이 노트북에서는 자연어 처리의 기본인 텍스트 벡터화를 직접 구현해봅니다.
- 텍스트 표준화 (Standardization)
- 토큰화 (Tokenization)
- 어휘 사전 구축 (Vocabulary Building)
- 인코딩/디코딩 (Encoding/Decoding)

## 학습 목표
1. 텍스트를 컴퓨터가 이해할 수 있는 숫자로 변환하는 과정 이해
2. 벡터화 과정의 각 단계별 역할 파악
3. 직접 구현을 통한 텍스트 전처리 원리 학습

## 1. 필요한 라이브러리 임포트

In [1]:
import string
print("구두점 목록:", string.punctuation)

구두점 목록: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


## 2. MyVectorize 클래스 구현

텍스트 벡터화를 위한 커스텀 클래스를 구현합니다.

In [2]:
class MyVectorize:
    """
    텍스트를 벡터로 변환하는 클래스
    
    주요 기능:
    - 텍스트 표준화 (소문자 변환, 구두점 제거)
    - 토큰화 (단어 단위로 분할)
    - 어휘 사전 구축
    - 텍스트 인코딩/디코딩
    """
    
    def standardize(self, text):
        """
        텍스트 표준화 함수
        
        Args:
            text (str): 입력 텍스트
            
        Returns:
            str: 표준화된 텍스트 (소문자 + 구두점 제거)
        """
        # 1. 전부 소문자로 변환
        text = text.lower()
        # 2. 구두점 제거
        return "".join(c for c in text if c not in string.punctuation)
    
    def tokenize(self, text):
        """
        토큰화 함수
        
        Args:
            text (str): 표준화된 텍스트
            
        Returns:
            list: 토큰(단어) 리스트
        """
        return text.split()  # 공백 기준으로 분할
    
    def make_vocabulary(self, dataset):
        """
        어휘 사전 구축 함수
        
        Args:
            dataset (list): 텍스트 데이터셋
            
        특수 토큰:
        - "": 패딩용 (인덱스 0)
        - "[UNK]": 미등록 단어용 (인덱스 1)
        """
        # 특수 토큰으로 초기화
        self.vocabulary = {"": 0, "[UNK]": 1}
        
        # 데이터셋 순회하며 어휘 사전 구축
        for text in dataset:
            # 표준화 -> 토큰화
            text = self.standardize(text)
            tokens = self.tokenize(text)
            
            # 새로운 토큰 발견시 사전에 추가
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        
        # 역방향 사전 생성 (숫자 -> 단어)
        self.inverse_vocabulary = dict((v, k) for k, v in self.vocabulary.items())
    
    def encode(self, text):
        """
        텍스트를 숫자 시퀀스로 인코딩
        
        Args:
            text (str): 인코딩할 텍스트
            
        Returns:
            list: 숫자 시퀀스
        """
        text = self.standardize(text)
        tokens = self.tokenize(text)
        # 사전에 없는 단어는 [UNK] 토큰(1)으로 처리
        return [self.vocabulary.get(token, 1) for token in tokens]
    
    def decode(self, int_sequence):
        """
        숫자 시퀀스를 텍스트로 디코딩
        
        Args:
            int_sequence (list): 숫자 시퀀스
            
        Returns:
            str: 디코딩된 텍스트
        """
        return " ".join(self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

## 3. 벡터화 클래스 테스트

구현한 클래스를 테스트 데이터로 검증해봅니다.

In [3]:
# 벡터화 인스턴스 생성
mv = MyVectorize()

# 테스트 데이터셋
dataset = [
    "I write, erase, reqrite",
    "Erase again, and then",
    "A poppy blooms",
    "Dog is pretty"
]

print("=== 테스트 데이터셋 ===")
for i, text in enumerate(dataset):
    print(f"{i+1}. {text}")

=== 테스트 데이터셋 ===
1. I write, erase, reqrite
2. Erase again, and then
3. A poppy blooms
4. Dog is pretty


### 3.1 표준화 및 토큰화 테스트

In [4]:
# 두 번째 문장으로 표준화/토큰화 테스트
test_sentence = dataset[1]
print(f"원본 문장: '{test_sentence}'")

# 표준화 테스트
standardized = mv.standardize(test_sentence)
print(f"표준화 결과: '{standardized}'")

# 토큰화 테스트
tokens = mv.tokenize(standardized)
print(f"토큰화 결과: {tokens}")

원본 문장: 'Erase again, and then'
표준화 결과: 'erase again and then'
토큰화 결과: ['erase', 'again', 'and', 'then']


### 3.2 어휘 사전 구축

In [5]:
# 어휘 사전 구축
mv.make_vocabulary(dataset)

print("=== 어휘 사전 (단어 -> 인덱스) ===")
for word, idx in mv.vocabulary.items():
    print(f"'{word}' -> {idx}")

print("\n=== 역방향 사전 (인덱스 -> 단어) ===")
for idx, word in mv.inverse_vocabulary.items():
    print(f"{idx} -> '{word}'")

print(f"\n총 어휘 개수: {len(mv.vocabulary)}")

=== 어휘 사전 (단어 -> 인덱스) ===
'' -> 0
'[UNK]' -> 1
'i' -> 2
'write' -> 3
'erase' -> 4
'reqrite' -> 5
'again' -> 6
'and' -> 7
'then' -> 8
'a' -> 9
'poppy' -> 10
'blooms' -> 11
'dog' -> 12
'is' -> 13
'pretty' -> 14

=== 역방향 사전 (인덱스 -> 단어) ===
0 -> ''
1 -> '[UNK]'
2 -> 'i'
3 -> 'write'
4 -> 'erase'
5 -> 'reqrite'
6 -> 'again'
7 -> 'and'
8 -> 'then'
9 -> 'a'
10 -> 'poppy'
11 -> 'blooms'
12 -> 'dog'
13 -> 'is'
14 -> 'pretty'

총 어휘 개수: 15


### 3.3 인코딩/디코딩 테스트

In [6]:
# 인코딩 테스트
test_text = "I write erase"
encoded = mv.encode(test_text)
print(f"입력 텍스트: '{test_text}'")
print(f"인코딩 결과: {encoded}")

# 각 단어의 인코딩 과정 상세 분석
print("\n=== 인코딩 과정 상세 분석 ===")
standardized_test = mv.standardize(test_text)
tokens_test = mv.tokenize(standardized_test)
for token in tokens_test:
    idx = mv.vocabulary.get(token, 1)
    print(f"'{token}' -> {idx}")

입력 텍스트: 'I write erase'
인코딩 결과: [2, 3, 4]

=== 인코딩 과정 상세 분석 ===
'i' -> 2
'write' -> 3
'erase' -> 4


In [7]:
# 디코딩 테스트
test_sequence = [2, 3, 4, 23]  # 일부는 존재하지 않는 인덱스
decoded = mv.decode(test_sequence)
print(f"입력 시퀀스: {test_sequence}")
print(f"디코딩 결과: '{decoded}'")

# 각 인덱스의 디코딩 과정 상세 분석
print("\n=== 디코딩 과정 상세 분석 ===")
for idx in test_sequence:
    word = mv.inverse_vocabulary.get(idx, "[UNK]")
    print(f"{idx} -> '{word}'")

입력 시퀀스: [2, 3, 4, 23]
디코딩 결과: 'i write erase [UNK]'

=== 디코딩 과정 상세 분석 ===
2 -> 'i'
3 -> 'write'
4 -> 'erase'
23 -> '[UNK]'


## 4. 전체 데이터셋 처리 결과 분석

In [8]:
print("=== 전체 데이터셋 인코딩 결과 ===")
for i, text in enumerate(dataset):
    encoded = mv.encode(text)
    decoded = mv.decode(encoded)
    print(f"\n{i+1}. 원본: '{text}'")
    print(f"   표준화: '{mv.standardize(text)}'")
    print(f"   인코딩: {encoded}")
    print(f"   디코딩: '{decoded}'")

=== 전체 데이터셋 인코딩 결과 ===

1. 원본: 'I write, erase, reqrite'
   표준화: 'i write erase reqrite'
   인코딩: [2, 3, 4, 5]
   디코딩: 'i write erase reqrite'

2. 원본: 'Erase again, and then'
   표준화: 'erase again and then'
   인코딩: [4, 6, 7, 8]
   디코딩: 'erase again and then'

3. 원본: 'A poppy blooms'
   표준화: 'a poppy blooms'
   인코딩: [9, 10, 11]
   디코딩: 'a poppy blooms'

4. 원본: 'Dog is pretty'
   표준화: 'dog is pretty'
   인코딩: [12, 13, 14]
   디코딩: 'dog is pretty'


## 5. 미등록 단어(UNK) 처리 테스트

In [9]:
# 어휘 사전에 없는 새로운 단어가 포함된 문장 테스트
new_text = "I love programming and coding"
print(f"새로운 텍스트: '{new_text}'")

# 표준화 및 토큰화
standardized_new = mv.standardize(new_text)
tokens_new = mv.tokenize(standardized_new)
print(f"토큰들: {tokens_new}")

# 각 토큰이 어휘 사전에 있는지 확인
print("\n=== 토큰별 어휘 사전 존재 여부 ===")
for token in tokens_new:
    if token in mv.vocabulary:
        print(f"'{token}' -> 등록됨 (인덱스: {mv.vocabulary[token]})")
    else:
        print(f"'{token}' -> 미등록 ([UNK]로 처리)")

# 인코딩 결과
encoded_new = mv.encode(new_text)
decoded_new = mv.decode(encoded_new)
print(f"\n인코딩 결과: {encoded_new}")
print(f"디코딩 결과: '{decoded_new}'")

새로운 텍스트: 'I love programming and coding'
토큰들: ['i', 'love', 'programming', 'and', 'coding']

=== 토큰별 어휘 사전 존재 여부 ===
'i' -> 등록됨 (인덱스: 2)
'love' -> 미등록 ([UNK]로 처리)
'programming' -> 미등록 ([UNK]로 처리)
'and' -> 등록됨 (인덱스: 7)
'coding' -> 미등록 ([UNK]로 처리)

인코딩 결과: [2, 1, 1, 7, 1]
디코딩 결과: 'i [UNK] [UNK] and [UNK]'


## 6. 결론 및 정리

### 구현한 벡터화 시스템의 특징:

1. **표준화 (Standardization)**
   - 대소문자 통일 (소문자 변환)
   - 구두점 제거로 일관성 확보

2. **토큰화 (Tokenization)**
   - 공백 기준 단어 분할
   - 간단하지만 효과적인 방법

3. **어휘 사전 (Vocabulary)**
   - 특수 토큰 활용 ("", "[UNK]")
   - 동적 사전 구축
   - 양방향 매핑 지원

4. **인코딩/디코딩**
   - 텍스트 ↔ 숫자 시퀀스 변환
   - 미등록 단어 처리 (UNK 토큰)
   - 정보 손실 최소화

### 실제 응용에서의 고려사항:
- 더 정교한 토큰화 (서브워드, BPE 등)
- 빈도 기반 어휘 필터링
- 패딩 및 시퀀스 길이 관리
- 대용량 데이터 처리 최적화

## 7. 추가 실습 문제

다음 문제들을 해결해보며 이해도를 높여보세요:

1. 한국어 텍스트로 동일한 과정을 수행해보기
2. 빈도수 기반 어휘 필터링 기능 추가하기
3. 최대 시퀀스 길이 제한 및 패딩 기능 구현하기
4. 문자 단위 토큰화와 단어 단위 토큰화 비교하기